# Counting issues by denominations

This notebook computes the number of issues by denomination for more than 50,000 records in OCRE.

## Organization of this notebook

This notebook uses Scala with the almond kernel (<https://almond.sh/>).  The cells labelled "Notebook configuration" configure the almond kernel to find and import a series of custom libraries using syntax specific to the ammonite shell that almond use.  This is analogous to defining imports in a `build.sbt` file if you were using `sbt` to run scala.

The following section (labelled  "Analyis with generic Scala") consists of completely generic scala that could be used in any environment with access to the repositories and libraries configured in the section labelled "Notebook configuration".


## Notebook configuration

Set up notebook for access to libraries.  For reasons I don't understand (but perhaps having to do with asynchronous loading) I have to separate out the two steps of adding a maven repository and using `$ivy` imports with those repositories into separate notebook cells.

In [27]:
// 1. Add maven repository where we can find our libraries
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

In [28]:
// 2. Make libraries available with `$ivy` imports:
import $ivy.`edu.holycross.shot::nomisma:0.4.0`
import $ivy.`edu.holycross.shot::histoutils:2.2.0`
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`


import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                                      


## Analyis with generic Scala

Import libraries, as always in Scala:

In [29]:
import edu.holycross.shot.nomisma._
import edu.holycross.shot.histoutils._
import scala.io.Source
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


import edu.holycross.shot.nomisma._

import edu.holycross.shot.histoutils._

import scala.io.Source

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


### Build an `Ocre` object

The following cell loads data in CEX format for:

- basic data about each issue
- texts of obverse and reverse legends
- text of OCRE description of obverse and reverse types
- geographic coordinate data for mints in OCRE

It unifies these in a single `Ocre` object 

In [30]:
val basicsUrl = "https://github.com/neelsmith/nomisma/raw/master/shared/src/test/resources/cex/ocre-basic-issues.cex"
// Drop header line
val data = Source.fromURL(basicsUrl).getLines.toVector.tail

val basics = for (cex <- data) yield {
    BasicIssue(cex)
}
val basicIssues = basics.toVector


val legendUrl = "https://raw.githubusercontent.com/neelsmith/nomisma/master/shared/src/test/resources/cex/ocre-legends.cex"
val legendList = for (cex <- Source.fromURL(legendUrl).getLines) yield {
    edu.holycross.shot.nomisma.Legend(cex)
}
val legends = legendList.toVector.flatten

val typesUrl = "https://raw.githubusercontent.com/neelsmith/nomisma/master/shared/src/test/resources/cex/ocre-types.cex"
val typeList = for (cex <- Source.fromURL(typesUrl).getLines) yield {
    TypeDescription(cex)
}
val typeVector = typeList.toVector.flatten

val geoUrl  = "https://raw.githubusercontent.com/neelsmith/nomisma/master/shared/src/test/resources/cex/mintgeo.cex"
val geoLines = Source.fromURL(geoUrl).getLines.toVector.tail
val mintPoints = geoLines.map(MintPoint(_)).toVector

// The composite Ocre object:
val ocre = Ocre(basicIssues, legends, typeVector, Vector.empty[Portrait], MintPointCollection(mintPoints))




basicsUrl: String = "https://github.com/neelsmith/nomisma/raw/master/shared/src/test/resources/cex/ocre-basic-issues.cex"
data: Vector[String] = Vector(
  "1_2.aug.10#RIC I (second edition) Augustus 10#denarius#ar#augustus#emerita#lu...
basics: Vector[BasicIssue] = Vector(
  BasicIssue(
...
basicIssues: Vector[BasicIssue] = Vector(
  BasicIssue(
...
legendUrl: String = "https://raw.githubusercontent.com/neelsmith/nomisma/master/shared/src/test/resources/cex/ocre-legends.cex"
legendList: Iterator[Vector[edu.holycross.shot.nomisma.Legend]] = empty iterator
legends: Vector[edu.holycross.shot.nomisma.Legend] = Vector(
  Legend("ID", Obverse, "ObvLegend"),
...
typesUrl: String = "https://raw.githubusercontent.com/neelsmith/nomisma/master/shared/src/test/resources/cex/ocre-types.cex"
typeList: Iterator[Vector[TypeDescription]] = empty iterator
typeVector: Vector[TypeDescription] = Vector(
  TypeDescription("ID", Obverse, "ObvType"),
...
geoUrl: String = "https://raw.githubusercontent.com/nee

### Compute and plot histogram by denomination

Settings for reasonable display using plotly:

In [31]:
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [33]:
val denomFreqs = ocre.issues.map(_.denomination).groupBy(d => d).map { case (k,v) => Frequency(k, v.size)}
val denomHisto = edu.holycross.shot.histoutils.Histogram(denomFreqs.toVector)

val denoms = denomHisto.sorted.frequencies.map(_.item)
val counts = denomHisto.sorted.frequencies.map(_.count)
val plottable = Seq(
  Bar(
   denoms, counts
  )
)

plot(plottable)




denomFreqs: collection.immutable.Iterable[Frequency[String]] = List(
  Frequency("quadrans", 262),
...
denomHisto: edu.holycross.shot.histoutils.Histogram[String] = Histogram(
  Vector(
...
denoms: Vector[String] = Vector(
  "antoninianus",
...
counts: Vector[Int] = Vector(
  9472,
...
plottable: Seq[Bar] = List(
  Bar(
...
res32_5: String = "plot-e303b911-ea6c-4bd6-8ca6-69471df7f79d"